In [45]:
import os
import random as rnd
import pandas as pd 
import math as mt

import pgutils as pg
import pgmath as pm
import algorithm as al
import numpy as np

In [46]:
root_dir = r'C:\Users\eleon\PycharmProjects\SM_Exam\data\yes_small'
#root_dir = './data/yes_small'

In [47]:
#test_data=[]
with open(os.path.join(root_dir, 'test.txt')) as f:
#with open(root_dir + "\\test.txt", "r") as f:

    test_data = f.readlines()
    
test_dataset = pg.data_to_list(test_data[2:])

In [48]:
#train_data=[]
with open(os.path.join(root_dir,"train.txt"), "r") as f:
    train_data = f.readlines()
    
#train_dataset = pg.data_to_list(train_data[2:])

In [49]:
song_hash = pd.read_csv(os.path.join(root_dir,"song_hash.txt"), sep="\t", header=None)


In [50]:
# setting parameters
dimension = 2#rnd.choice([2, 5,10,25,50,100])
# regularization parameter set by cross validation
lam = rnd.choice([0.0001, 0.001, 0.01, 0.1, 1, 10, 20, 50, 100, 500, 1000])
# regularization parameter for dual point model
nu = rnd.choice([0, lam])
# threshold for landmark dimension
r = 0.2
# number of landmark
n_landmarks = 3#50
# num iteration 100 o 200
n_iter = 60 #rnd.choice([100, 200])
# tau predefined learning rate
tau = 0.5

In [51]:
toy_dataset=[[0,6,3,2,1,4,2],[2,0,6,5,6,3,0],[0,5,2,1,6],[5,6,2],[6,5,2,3,5,6,2]]

songs = 7
transition_matrix = pg.transition_count(songs, toy_dataset)
position = np.random.rand(songs, dimension)
num_transition = np.sum(transition_matrix)
params = pm.AlgParams(lam, nu, tau, num_transition, n_landmarks, r, dimension, n_iter)

In [52]:
X = al.single_point_algorithm(songs, transition_matrix, params)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59


In [87]:
dummy_landmarks = [[i for i in range(songs)] for j in range(songs)]
d2 = pm.Distances.delta(X)
prob_matrix = np.exp(-d2)/pm.Distances.zeta(d2,X,dummy_landmarks)
cum_sum = np.sum(prob_matrix, axis=1)
prob_matrix = prob_matrix / cum_sum[:,np.newaxis]

In [84]:
print('cum',cum_sum)
print('sum norm', np.sum(prob_matrix, axis=1))
#print('sum row ',np.sum(prob_matrix/cum_sum[0], axis=0))
print(np.sum(prob_matrix[1]))

cum [0.84609751 0.9646615  0.82874336 0.88572032 0.96488818 0.7641816
 0.82577392]
sum norm [1. 1. 1. 1. 1. 1. 1.]
1.0


In [85]:
print(prob_matrix)
print(np.sum(prob_matrix, axis=0))

[[0.23798622 0.11838589 0.09615957 0.09495869 0.12057908 0.22231058
  0.10961998]
 [0.12348675 0.17551827 0.13813512 0.14024347 0.16939499 0.12061792
  0.13260349]
 [0.09846939 0.13561022 0.24223885 0.17073142 0.13118685 0.10052516
  0.12123811]
 [0.09841321 0.13934163 0.17279191 0.20954666 0.13685519 0.0957732
  0.1472782 ]
 [0.12594767 0.1696283  0.13381345 0.13793064 0.1751945  0.12218599
  0.13529944]
 [0.2015581  0.10484109 0.08900331 0.08378484 0.10605798 0.32178023
  0.09297445]
 [0.1142824  0.13253305 0.12342982 0.14815266 0.13504156 0.10690872
  0.23965178]]
[1.00014374 0.97585845 0.99557204 0.98534837 0.97431015 1.09010179
 0.97866546]


In [86]:
toy_test=[[0,4,5,3,6],[4,3,2,6],[0,2,1,0,5]]
# evaluate test performance using the average log-likelihood as metric
# it is defined as log(Pr(d_test))/n_test)
# n_test number of transition in the test set
songs = 7 #len(song_hash)
n_test = np.sum(pg.transition_count(songs,toy_test))
print(n_test)

11.0


In [57]:
def log_like(test_set, probability_matrix):
    count = 0
    for i in range(len(test_set)):
        for predecessor, successor in zip(test_set[i][:-1], test_set[i][1:]):
            count = count + mt.log(probability_matrix[predecessor, successor])

    return count
evaluation = log_like(toy_test,prob_matrix) /n_test
print(evaluation)
#result = sum( sum(dimension.D[s, i.index()] ) for i in range(len(test_dataset)))  

-2.0337920532306204


In [58]:
#generate a playlist
#from __future__ import division
#import matplotlib.pyplot as plt
#%matplotlib inline

In [135]:
#The value of the process are stored in a list X
#The list of songs
Songs=range(7)

In [136]:
def playlist_generator(num_song, current_song):
    #Start of the time
    t=0
    #The music playlist
    playlist=[current_song]
    #initial probability
    prob = 1
    #while loop - the Markov chain did not reach 5 we go on
    while t < num_song:
        #Incrementation of time
        t=t+1
        #List of played songs
        playlist.append(np.random.choice(Songs, replace=True, p=prob_matrix[current_song]))
        prob = prob * prob_matrix[current_song, playlist[-1]]
        current_song = playlist[-1]

    #Printing the path of the Markov chain
    #Printing the number of steps
    #print(len(X)-1)
    titles = [song_hash.iloc[i][1] for i in playlist]
    print("Song in the playlist: ", titles)
    print("End state after ", num_song , " days: ", song_hash.iloc[current_song][1])
    print("Probability of the possible sequence of states: ", str(prob))

In [137]:
# Function that forecasts the possible state for the next 2 days
playlist_generator(400, 5)






Song in the playlist:  ['Down On Me (w\\/ 50 Cent)', 'Aston Martin Music (w\\/ Drake & Chrisette Michelle)', 'Down On Me (w\\/ 50 Cent)', 'Black And Yellow', 'Get Back Up (w\\/ Chris Brown)', 'Get Back Up (w\\/ Chris Brown)', 'Whip My Hair', 'Whip My Hair', 'Down On Me (w\\/ 50 Cent)', 'Down On Me (w\\/ 50 Cent)', 'Down On Me (w\\/ 50 Cent)', 'Gucci Time (w\\/ Swizz Beatz)', 'Down On Me (w\\/ 50 Cent)', 'Aston Martin Music (w\\/ Drake & Chrisette Michelle)', 'Gucci Time (w\\/ Swizz Beatz)', 'Whip My Hair', 'Hot Toddy (w\\/ Jay-Z & Ester Dean)', 'Down On Me (w\\/ 50 Cent)', 'Get Back Up (w\\/ Chris Brown)', 'Get Back Up (w\\/ Chris Brown)', 'Gucci Time (w\\/ Swizz Beatz)', 'Black And Yellow', 'Down On Me (w\\/ 50 Cent)', 'Gucci Time (w\\/ Swizz Beatz)', 'Gucci Time (w\\/ Swizz Beatz)', 'Down On Me (w\\/ 50 Cent)', 'Down On Me (w\\/ 50 Cent)', 'Whip My Hair', 'Down On Me (w\\/ 50 Cent)', 'Down On Me (w\\/ 50 Cent)', 'Aston Martin Music (w\\/ Drake & Chrisette Michelle)', 'Black And Yello